In [16]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import pickle
import os
import multiprocessing
from PF_Aso1_0_CPU import *

workdir = os.path.dirname(os.getcwd())
srcdir = os.getcwd()
datadir = workdir + '/data/'
outputdir = workdir + '/output/'

In [17]:
seed = 7

obs_series = pd.read_csv(datadir + 'data.csv', delimiter=',')
obs_series = np.array(obs_series.iloc[:,1:]).T

T = obs_series.shape[1]
N = 10000
Λ_scale = 1.0
cd_scale = 1.0

case = 'actual data, seed = ' + str(seed) + ', T = ' + str(T) + ', N = ' + str(N) + ', Λ_scale = ' + str(Λ_scale) + ', cd_scale = ' + str(cd_scale)
try: 
    casedir = outputdir + case  + '/'
    os.mkdir(casedir)
except:
    casedir = outputdir + case  + '/'

np.random.seed(seed)
start_time = time.time()

D_0 = obs_series[:,[0]]

Input_0 = [[D_0, Λ_scale, cd_scale] for i in range(N)]
pool = multiprocessing.Pool()
Output_0 = pool.map(init, tqdm(Input_0))
del(Input_0)
θ_t_particle = [i[0] for i in Output_0]
X_t_particle = [i[1] for i in Output_0]
H_t_particle = [i[2] for i in Output_0]
del(Output_0)
with open(casedir + 'θ_0.pkl', 'wb') as f:
    pickle.dump(θ_t_particle, f)
del(θ_t_particle)
# with open(casedir + 'X_0.pkl', 'wb') as f:
#     pickle.dump(X_t_particle, f)
# with open(casedir + 'H_0.pkl', 'wb') as f:
#     pickle.dump(H_t_particle, f)
with open(casedir + 'count_0.pkl', 'wb') as f:
    pickle.dump(list(np.ones(N)), f)
with open(casedir + 'w_0.pkl', 'wb') as f:
    pickle.dump(list(np.ones(N)/N), f)
run_time = time.time() - start_time
print(run_time) 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:07<00:00, 1290.57it/s]


12.588054180145264


In [18]:
def draw_para(b, Λ, c, d):
    ζ = sp.stats.gamma.rvs(c/2+1, loc = 0, scale = 1/(d/2))
    σ2 = 1/ζ
    β = np.random.multivariate_normal(b.flatten(), sp.linalg.inv(ζ*Λ)).reshape(-1,1)
    return β, σ2

In [19]:
b = [H_t_particle[i][0][0] for i in range(len(H_t_particle))]
Λ = [H_t_particle[i][0][1] for i in range(len(H_t_particle))]
c = [H_t_particle[i][0][2] for i in range(len(H_t_particle))]
d = [H_t_particle[i][0][3] for i in range(len(H_t_particle))]

In [22]:
%%time
ζ = sp.stats.gamma.rvs(np.array(c)/2+1, loc = 0, scale = 1/(np.array(d)/2))
σ2 = 1/ζ
mean = np.array(b)
cov = np.multiply(np.repeat(ζ,2).reshape(-1,1),sp.linalg.block_diag(*Λ))

CPU times: user 626 ms, sys: 1.11 s, total: 1.74 s
Wall time: 2.13 s


In [27]:
β = np.random.multivariate_normal(mean.flatten(), cov).reshape(-1,1)

In [43]:
ζ.shape

(100000,)

In [30]:
50 in [50,100,150,200,250]

True

In [28]:
sp.linalg.block_diag(Λ[i] for i in range(len(Λ)))

array([[<generator object <genexpr> at 0x7fda07a8dc80>]], dtype=object)